In [1]:
import tensorflow as tf
import numpy as np
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

# 导入所有必要的层

import os
import keras
import numpy as np
import tensorflow.keras.applications.mobilenet_v2
from tensorflow.keras import layers
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import CSVLogger

In [ ]:
# Sequence
num_classes = 19
input_shape = (5, 60, 60, 1)

def split_train_and_test(x, y, test_size):
    x_train, x_test, y_train, y_test = [], [], [], []
    for i in range(y.max()+1):
        all_x = x[y==i]
        shuf = np.random.permutation(len(all_x))
        all_x = all_x[shuf]
        percent = int(len(all_x)*(1-test_size))
        x_train.extend(all_x[:percent])
        x_test.extend(all_x[percent:])
        y_train.extend([i]*percent)
        y_test.extend([i]*(len(all_x)-percent))
    x_train = np.array(x_train)
    x_test = np.array(x_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    return x_train, x_test, y_train, y_test


x = np.load('./train_unsw_60_60.npy')
y = np.load('./label_unsw_60_60.npy')
x_train, x_test, y_train, y_test = split_train_and_test(x, y, test_size=0.3)
x_raw = np.load('./train_unsw_60_60_raw.npy')
y_raw = np.load('./train_unsw_60_60_raw.npy')

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
x_raw = x_raw.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
x_raw = np.expand_dims(x_raw, -1)

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")
print(x_raw.shape[0], "raw samples")

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
x_raw.shape

In [ ]:
class InvertedBottleNeckLayers(tf.keras.Model):
    def __init__(self,in_fitler,expansion_rate,stride):
        super(InvertedBottleNeckLayers, self).__init__()
        #self.in_filter=in_fitler
        self.stride=stride
        self.expansion_rate=expansion_rate

        self.conv1=layers.Conv2D(expansion_rate*in_fitler,kernel_size=[1,1],strides=[1,1],padding='same')
        self.conv1Batch=layers.BatchNormalization()
        self.conv1relu6=layers.Activation('relu6')

        self.Dwise=layers.DepthwiseConv2D(kernel_size=[3,3],strides=stride,padding='same')
        self.Dwisebatch=layers.BatchNormalization()
        self.Dwiserelu6=layers.Activation('relu6')

        self.conv22 = layers.Conv2D(in_fitler, kernel_size=[1, 1], strides=[1, 1], padding='same')
        self.conv22batch = layers.BatchNormalization()

    def call(self,inputs,training=None):
        
        if len(tf.shape(inputs)) == 5:
            batch = tf.shape(inputs)[0]
            height = tf.shape(inputs)[2]
            width = tf.shape(inputs)[3]
            channel = tf.shape(inputs)[4]
            inputs = tf.reshape(inputs, [batch*5, height, width,channel])
        
        x=self.conv1(inputs)
        x=self.conv1Batch(x)
        x=self.conv1relu6(x)

        x=self.Dwise(x)
        x=self.Dwisebatch(x)
        x=self.Dwiserelu6(x)
        DwiseChannel=np.shape(x)[-1]
        
        conv22batch = self.conv22batch(x)
        #如果步长为1且经过深度卷积输出的通道数和输入的通道数相同，
        #则进行shortcut，否则直接输出
        #conv22 = layers.Conv2D(self.in_filter, kernel_size=[1, 1], strides=[1, 1], padding='same')(x)
        #conv22batch = layers.BatchNormalization()(conv22)
        if self.stride==1 and DwiseChannel==np.shape(inputs)[-1]:
            x_out=tf.add(conv22batch,inputs)
        else:
            x_out=conv22batch
        #print(x_out)
        return x_out

class AutoFPExtractNet(tf.keras.Model):
    def __init__(self,in_fitler=32,num_classes=19):
        super(AutoFPExtractNet, self).__init__()
        self.conv11=layers.Conv2D(in_fitler,kernel_size=[3,3],strides=[2,2],padding='same')
        self.conv11batch=layers.BatchNormalization()
        self.conv11relu=layers.Activation('relu6')

        #self.bottleneck(t,c,n,c,i)这里的i表示第几个bottleneck
        self.bottleneck1 = self.bottleneck(1, 16, 1, 1, 1)
        self.bottleneck2 = self.bottleneck(1, 24, 2, 2, 2)
        self.bottleneck3 = self.bottleneck(6, 32, 2, 2, 3)
        self.bottleneck4 = self.bottleneck(6, 48, 3, 2, 4)
        self.bottleneck5 = self.bottleneck(6, 64, 1, 1, 5)

        self.conv22=layers.Conv2D(192,kernel_size=[1,1],strides=[1,1],padding='same')
        self.conv22batch=layers.BatchNormalization()
        self.conv22relu=layers.Activation('relu6')
        
        self.dropout=layers.Dropout(0.5)
        self.gru=layers.GRU(64, return_sequences=False)
        self.l2normalize=layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))


    def bottleneck(self,t,c,n,s,i):
        bottle=keras.Sequential([],name='bottleneck'+str(i))
        bottle.add(InvertedBottleNeckLayers(c,t,stride=s))
        for i in range(n-1):
            bottle.add(InvertedBottleNeckLayers(c,t,1))
        return bottle

    def call(self,inputs,training=None):
        x = self.conv11(inputs)
        x = self.conv11batch(x)
        x = self.conv11relu(x)

        x = self.bottleneck1(x)
        x = self.bottleneck2(x)
        x = self.bottleneck3(x)
        x = self.bottleneck4(x)
        x = self.bottleneck5(x)
        
        x = self.conv22(x)
        x = self.conv22batch(x)
        x = self.conv22relu(x)
        
        x = self.dropout(x)
        height = tf.shape(x)[1]
        width = tf.shape(x)[2]
        channel = tf.shape(x)[3]
        x = tf.reshape(x, [-1, 5, height * width * channel])
        x = self.gru(x)
        x = self.l2normalize(x)

        return x

model=AutoFPExtractNet()
model.build(input_shape=(None, None, 60, 60, 1))
model.call(tf.keras.layers.Input(shape=(None, 60, 60, 1),batch_size = 128))

In [ ]:
model.summary()

In [ ]:
batch_size = 128
epochs = 200
model.compile(loss=tfa.losses.TripletSemiHardLoss(margin=0.5), optimizer=tf.keras.optimizers.Adam(0.001))
# 日志记录
csv_logger = CSVLogger('./dataset2_training_dscgru_ex1.csv',append=False)
# 训练模型
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[csv_logger])

In [9]:
# results = model.predict(x_test)
results = model.predict(x_train)

In [ ]:
results.shape

In [ ]:
y_train

In [ ]:
np.save('/home/fengcorn/code/SeqTGI-IoTID/Ex2_dscgru_predict_result.npy', results)
test_results = model.predict(x_raw)
np.save('/home/fengcorn/code/SeqTGI-IoTID/Ex2_dscgru_predict_result_raw.npy', test_results)
np.save('/home/fengcorn/code/SeqTGI-IoTID/Ex2_dscgru_y_raw.npy', y_raw)
np.save('/home/fengcorn/code/SeqTGI-IoTID/Ex2_dscgru_y_train.npy', y_train)